In [1]:
import scipy.io
import numpy as np
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib
from astropy.table import Table
from astropy.coordinates import EarthLocation
import astropy
from astropy.time import Time
import astropy.units as u
data=scipy.io.loadmat('BNS-sources-trial-10saleem.mat')['data'] 
#sourceparameters
DL=data[:,0]
theta=data[:,1]
phi=data[:,2]

Using matplotlib backend: MacOSX


In [2]:
def radectolatlong(Ra,Dec,time): #input in degrees
    ra = np.deg2rad(Ra)
    dec = np.deg2rad(Dec)
    time_sid = (time.sidereal_time('mean','greenwich')).rad
    lat = dec
    long = ra - time_sid 
    Lat = np.rad2deg(lat)
    Long = np.rad2deg(long)-100.46
    return Lat,Long #output in degrees

def latlongtoradec(Lat,Long,time): #input in degrees
    lat = np.deg2rad(Lat)
    long = np.deg2rad(Long)
    time_sid = (time.sidereal_time('mean','greenwich')).rad
    
    DEC = lat
    RA = long + time_sid
    ra = np.rad2deg(RA)+100.46       #The value of 100.46 degrees is the value needed to make the -
                                      #expression yield the correct value for GMST at 0h UT on 1 January 2000.
    dec = np.rad2deg(DEC)
    return ra,dec #output in degrees



def getxyz(theta,phi):
    #if n==1:
        #lat, long
    theta = np.pi/2 - theta
    x=np.sin(theta)*np.cos(phi)
    y=np.sin(theta)*np.sin(phi)
    z=np.cos(theta)
    return x,y,z
def getnorm(vec):
    norm = np.linalg.norm(vec)
    #print (norm)
    return vec/norm

In [3]:
def randomlumino(number):
    """
    Returns 'number' of Luminosity values between the given caps
    """
    lumino = []
    lummax = -6
    lummin = -7
    factor = 10**6
    for i in range(number):
        #case1
        lumino.append(10**(random.randint(lummin*factor,lummax*factor)/factor))
        lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         #case2
#         lumino.append(10**lummax)
#         lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         #case3
#         if DL[i]<=200:
#             lumino.append(10**(random.randint(lummin*factor,lummax*factor)/factor))
#             lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         else:
#             lumino.append(0.0)
#             lumino[i] = 0
#         #case4
#         if DL[i]<=200:
#             lumino.append(10**(lummin))
#             lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         else:
#             lumino.append(0.0)
#             lumino[i] = 0
    return lumino
    
#         if DL[i]<=200:
#             lumino.append(10**(random.randint(lummin*factor,lummax*factor)/factor))
# #             lumino.append(10**(lummin))
#             lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
#         else:
#             lumino.append(0.0)
# #             lumino[i] = 0
# #         lumino[i] = lumino[i]*(200**2)/((DL[i])**2)
# #         lumino.append(10**lummax)
        
#     return lumino

lumino = randomlumino(len(homotime))
# len(homotime)
# print(lumino)

NameError: name 'homotime' is not defined

In [3]:
sensfile = np.loadtxt('/Users/jinaypatel/Desktop/Daksha/sensitivity_20det.csv', delimiter=',',skiprows=1)

In [ ]:
def transform(astrosat):   
#transforming coordinate system to satellite's frame.
    thetatransarray = []
    phitransarray = []
    for i in range(len(theta)):
        if i%10000==0:
            print(i, end=' ')
        astrosat.compute(homotime[i])
        satvecold = getnorm(getxyz(astrosat.sublat,astrosat.sublong,1))
        tplus = homotime[i][0:17]+(str(int(homotime[i][17:19])+1)) + homotime[i][19:23]
        astrosat.compute(tplus)
        satvecplus = getnorm(getxyz(astrosat.sublat,astrosat.sublong,1))

        zaxis = getnorm(satvecold)
        xaxistemp = getnorm([satvecplus[0]-satvecold[0],satvecplus[1]-satvecold[1],satvecplus[2]-satvecold[2]])
        yaxis = getnorm(np.cross(zaxis,xaxistemp))
        xaxis = getnorm(np.cross(yaxis,zaxis))

        xtrig,ytrig,ztrig = getxyz(theta[i],phi[i],0)
        trigvec = getnorm([xtrig,ytrig,ztrig])

        thetatrans = np.arccos(np.dot(trigvec,zaxis))
        yprojtrig = np.linalg.norm(trigvec)*np.dot(trigvec,yaxis)
        xprojtrig = np.linalg.norm(trigvec)*np.dot(trigvec,xaxis)
        phitrans = np.arctan2(yprojtrig, xprojtrig)
        thetatransarray.append(thetatrans)
        phitransarray.append(phitrans)
    return thetatransarray,phitransarray

theta1,phi1 = transform(astrosat1)
theta2,phi2 = transform(astrosat2)



In [ ]:
sensitivitydata1 = []
sensitivitydata2 = []
sensfilethetaphi = []
for i in range(len(sensfile)):
    sensfilethetaphi.append((sensfile[i][0], sensfile[i][1]))
    

In [ ]:
for j in range(len(theta)):
    if j%100==0:
        print(int(j*100/len(theta)),'% Completed', end='\r')
    vectrig = [theta1[j],phi1[j]]
    nearest = sensfilethetaphi[spatial.KDTree(sensfilethetaphi).query(vectrig)[1]]
    for i in range(len(sensfilethetaphi)):
        if sensfilethetaphi[i][0] == nearest[0]:
            if sensfilethetaphi[i][1]==nearest[1]:
                index=i
    
    sensitivitydata1.append(sensfile[index][3])

for j in range(len(theta)):
    if j%100==0:
        print(int(j*100/len(theta)),'% Completed', end='\r')
    vectrig = [theta2[j],phi2[j]]
    nearest = sensfilethetaphi[spatial.KDTree(sensfilethetaphi).query(vectrig)[1]]
    for i in range(len(sensfilethetaphi)):
        if sensfilethetaphi[i][0] == nearest[0]:
            if sensfilethetaphi[i][1]==nearest[1]:
                index=i
    
    sensitivitydata2.append(sensfile[index][3])

In [1]:
for k in range(len(sensitivitydata2)):
    if np.isnan(sensitivitydata2[k])==True:
        sensitivitydata2[k] = 0
        
sensdatabackup2 = sensitivitydata2

for k in range(len(sensitivitydata1)):
    if np.isnan(sensitivitydata1[k])==True:
        sensitivitydata1[k] = 0
        
sensdatabackup1 = sensitivitydata1

NameError: name 'sensitivitydata2' is not defined

In [ ]:
numtrig = 0
for i in range(len(lumino)):
    if lumino[i]>0:numtrig = numtrig + 1
line1=[] #DD
line2=[] #LIGO
line3=[] #DD + LIGO
line4=[] #D : chosen without any reason as the first among these two (ephem = astrosat1)
line5=[] #D + LIGO
line6=[] #DD detections which LIGO didn't see
line7=[] #D detections which LIGO didn't see
print(numtrig)
# senssattemp = 1.4*10**(-7)
# senssattemp = 7.0*10**(-8)
for i in range(12):
    snr=i
    print('\nSNR =', snr)
    boolvis = []
    countsaa1, countsens1, countearth1, countvis1= 0,0,0,0
    countsaa2, countsens2, countearth2, countvis2 = 0,0,0,0
    count1sat,count1sat = 0,0
    count2sat,count2sat = 0,0
    count0sat,count0sat = 0,0
    vis1, vis2 = 0,0
    satd0, satd1, satd2 = 0,0,0
    dnol, ddnol0, ddnol1,ddnol2 = 0,0,0,0
    lowsnr = 0
    for i in range(len(homotime)):
        if i%10000==0:
            print(int(i*100/len(homotime)),'%',end='\r', flush=True)
        if snrLHVKI[i]<snr:
                lowsnr=lowsnr + 1
        if lumino[i]>0:
            satyes, yes, var3 = 0,0,0
            if is_saa(sublat1[i],sublong1[i], homotime[i])==True:
                countsaa1=countsaa1 + 1
            if sensitivitydata1[i]>lumino[i]:
                countsens1=countsens1+1
            if is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==False:
                countearth1 = countearth1 +1
            if is_saa(sublat1[i],sublong1[i], homotime[i])==False and snrLHVKI[i]>=snr and sensitivitydata1[i]<=lumino[i] and is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==True:
                countvis1 = countvis1+1
                satyes = satyes + 1
            if is_saa(sublat1[i],sublong1[i], homotime[i])==False and sensitivitydata1[i]<=lumino[i] and is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==True:
                vis1 = vis1+1
                yes = yes + 1
            if is_saa(sublat1[i],sublong1[i], homotime[i])==False and snrLHVKI[i]<snr and sensitivitydata1[i]<=lumino[i] and is_visible(homotime[i],sublat1[i],sublong1[i],np.deg2rad(90.0) - theta1[i],phi1[i], astrosat1)==True:
                var3=var3+1
                dnol=dnol+1
                
            if is_saa(sublat2[i],sublong2[i], homotime[i])==True:
                countsaa2=countsaa2 + 1
            if sensitivitydata2[i]>lumino[i]:
                countsens2=countsens2+1
            if is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat1)==False:
                countearth2 = countearth2 +1
            if is_saa(sublat2[i],sublong2[i], homotime[i])==False and snrLHVKI[i]>=snr and sensitivitydata2[i]<=lumino[i] and is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat2)==True:
                countvis2 = countvis2+1
                satyes = satyes + 1
            if is_saa(sublat2[i],sublong2[i], homotime[i])==False and sensitivitydata2[i]<=lumino[i] and is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat2)==True:
                vis2 = vis2+1
                yes = yes + 1           
            if is_saa(sublat2[i],sublong2[i], homotime[i])==False and snrLHVKI[i]<snr and sensitivitydata2[i]<=lumino[i] and is_visible(homotime[i],sublat2[i],sublong2[i],np.deg2rad(90.0) - theta2[i],phi2[i], astrosat2)==True:
                var3=var3+1

            if satyes==0:
                count0sat=count0sat+1
            if satyes==1:
                count1sat=count1sat+1
            if satyes==2:
                count2sat=count2sat+1
            if yes==0:
                satd0=satd0+1
            if yes==1:
                satd1=satd1+1
            if yes==2:
                satd2=satd2+1
            if var3==0:
                ddnol0=ddnol0+1
            if var3==1:
                ddnol1=ddnol1+1
            if var3==2:
                ddnol2=ddnol2+1
    
    print('***100% Completed***\n',end='\r',flush=True)        
    print('lowsnr', lowsnr,'\t', lowsnr*100/numtrig)
    print('goodsnr', numtrig - lowsnr,'\t', (numtrig - lowsnr)*100/numtrig)
    print('\nDL')
    print('countvis1', countvis1,'\t', countvis1*100/(numtrig-lowsnr))
    print('countvis2', countvis2,'\t', countvis2*100/(numtrig-lowsnr))
    print('oD')
    print('vis1', vis1,'\t', vis1*100/(numtrig))
    print('vis2', vis2,'\t', vis2*100/(numtrig))
    print('\nDL', count0sat, count1sat, count2sat)
    print('oD', satd0, satd1, satd2)
    print((count1sat+count2sat+count0sat)*100/numtrig, '-- should be equal to 100%\n')
    print((count1sat+count2sat), '\t',(count1sat+count2sat)*100/(numtrig-lowsnr) ,'(' + str(count1sat*100/numtrig+count2sat*100/numtrig)+ ')')
    print('Line 1 element(Seen by D-Daksha)                      ', satd1+satd2)
    print('Line 2 element(Seen by Ligo network)                  ', len(homotime)-lowsnr)
    print('Line 3 element(Seen by both D-Daksha & LIGO)          ', count1sat+count2sat)
    print('Line 4 element(Seen by only one Daksha)               ', vis1)
    print('Line 5 element(Seen by both Daksha & LIGO)            ', countvis1)
    print('Line 6 element(LIGO missed which D-Dakhsa detected)   ', ddnol1+ddnol2)
    print('Line 7 element(LIGO missed which Daksha detected)     ', dnol)
    line1.append(satd1+satd2)
    line2.append(len(homotime)-lowsnr)
    line3.append(count1sat+count2sat)
    line4.append(vis1)
    line5.append(countvis1)
    line6.append(ddnol1+ddnol2)
    line7.append(dnol)